In [4]:
import os
import json
import numpy as np
import pandas as pd
from scipy.signal import find_peaks

# === Extract Y values from specific landmarks ===
def extract_y_from_landmarks(landmarks, names):
    coords = []
    for name in names:
        for kp in landmarks:
            if kp["name"] == name and kp["conf"] > 0.5:
                coords.append(kp["y"])
                break
    return coords if len(coords) == len(names) else None

# === Count Squat Reps from Combined Y-Series ===
def count_squat_reps_from_json(json_path, min_prominence=0.02, min_distance_frames=20):
    import scipy.ndimage

    with open(json_path, 'r') as f:
        data = json.load(f)

    y_values = []
    required_landmarks = [
        "left_shoulder", "right_shoulder",
        "left_hip", "right_hip",
        "left_knee", "right_knee"
    ]

    for frame in data:
        lm = frame["landmarks"]
        coords = extract_y_from_landmarks(lm, required_landmarks)
        if coords:
            y_values.append(np.mean(coords))

    if len(y_values) < 10:
        return 0

    y_array = np.array(y_values)

    # --- Normalize and Smooth ---
    y_array = (y_array - np.min(y_array)) / (np.max(y_array) - np.min(y_array))  # Normalize
    y_array = scipy.ndimage.gaussian_filter1d(y_array, sigma=2)  # Smooth to reduce jitter

    # --- Peak & Valley Detection ---
    valleys, _ = find_peaks(-y_array, prominence=min_prominence, distance=min_distance_frames)
    peaks, _ = find_peaks(y_array, prominence=min_prominence, distance=min_distance_frames)

    # --- Match each up peak to prior down valley for valid rep ---
    rep_count = 0
    last_valley = -1
    for peak in peaks:
        valid_valleys = [v for v in valleys if last_valley < v < peak]
        if valid_valleys:
            rep_count += 1
            last_valley = valid_valleys[-1]

    return rep_count


# === Batch Process JSON Files in Folder ===
def process_squat_json_folder(folder_path, output_csv='squat_rep_counts_json.csv'):
    results = []
    json_files = [f for f in os.listdir(folder_path) if f.lower().endswith(".json")]

    for file in json_files:
        json_path = os.path.join(folder_path, file)
        try:
            reps = count_squat_reps_from_json(json_path)
            results.append({'Video_JSON': file, 'Rep_Count': reps})
            print(f"{file}: {reps} reps")
        except Exception as e:
            print(f"Error processing {file}: {e}")
            results.append({'Video_JSON': file, 'Rep_Count': 0})

    df = pd.DataFrame(results)
    output_path = os.path.join(folder_path, output_csv)
    df.to_csv(output_path, index=False)
    print(f"\n✅ Results saved to {output_path}")
    return df

# === Example Usage ===
if __name__ == "__main__":
    folder = r"C:\Users\bdsid\OneDrive\Desktop\Dune Tech\Rep Counting\Data\Kaggle\raw_data\data-btc\squat\Json"
    process_squat_json_folder(folder)


squat_0_pose.json: 5 reps
squat_10_pose.json: 1 reps
squat_1_pose.json: 1 reps

✅ Results saved to C:\Users\bdsid\OneDrive\Desktop\Dune Tech\Rep Counting\Data\Kaggle\raw_data\data-btc\squat\Json\squat_rep_counts_json.csv
